# Demo for Application Scorecard Model Documentation

This interactive notebook guides model developers through the process of buidling and documenting an application scorecard model with the ValidMind Developer Framework. It uses the [Lending Club](https://www.kaggle.com/datasets/devanshi23/loan-data-2007-2014/data) sample dataset from Kaggle to build a simple application scorecard.

As part of the notebook, you will learn how to train a sample model while exploring how the documentation process works:

- Initializing the ValidMind Developer Framework
- Loading a sample dataset provided by the library to train an application scorecard model
- Running a ValidMind test suite to quickly generate documention about the data and model

## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, you can make use of the following resources alongside this notebook:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) — The path for developers, more code samples, and our developer reference


## Before you begin

::: {.callout-tip}

### New to ValidMind?

For access to all features available in this notebook, create a free ValidMind account.

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:


In [ ]:
%pip install -q validmind

## Initialize the client library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the client library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/register-models-in-model-inventory.html))

   For example, to register a model for use with this notebook, select:

   - Documentation template: `Credit Risk Scorecard`
   - Use case: `Risk Management/CECL`

   You can fill in other options according to your preference.

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [ ]:
import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)

## Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:

In [ ]:
import statsmodels.api as sm

%matplotlib inline

### Preview the documentation template

A template predefines sections for your model documentation and provides a general outline to follow, making the documentation process much easier.

You will upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:

In [ ]:
vm.preview_template()

## Load the sample dataset

The sample dataset used here is provided by the ValidMind library. To be able to use it, you need to import the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), a two-dimensional tabular data structure that makes use of rows and columns:

In [ ]:
# Import the sample dataset from the library

from validmind.datasets.credit_risk import lending_club

df = lending_club.load_data(source="offline")

df.head()

### Prepocess the dataset

In the preprocessing step we performs a number of operations to get ready for building our application scorecard. We use the `lending_club.preprocess` to simplify preprocessing. This function performs the following operations: 

- Filters the dataset to include only loans for debt consolidation or credit card purposes.
- Removes loans classified under the riskier grades 'F' and 'G'.
- Excludes uncommon home ownership types and standardizes employment length and loan terms into numerical formats.
- Discards unnecessary fields and any entries with missing information to maintain a clean and robust dataset for modeling.

In [ ]:
preprocess_df = lending_club.preprocess(df)
preprocess_df.head()

### Feature engineering

In the feature engineering phase, we apply specific transformations to optimize the dataset for predictive modeling in our application scorecard. Using the l`ending_club.feature_engineering()` function, we conduct the following operations:

- **WoE encoding**: converts both numerical and categorical features into Weight of Evidence (WoE) values. WoE is a statistical measure used in scorecard modeling that quantifies the relationship between a predictor variable and the binary target variable. It calculates the ratio of the distribution of good outcomes to the distribution of bad outcomes for each category or bin of a feature. This transformation helps to ensure that the features are predictive and consistent in their contribution to the model.
- **Integration of WoE bins**: ensures that the WoE transformed values are integrated throughout the dataset, replacing the original feature values while excluding the target variable from this transformation. This transformation is used to maintain a consistent scale and impact of each variable within the model, which helps make the predictions more stable and accurate.

In [ ]:
fe_df = lending_club.feature_engineering(preprocess_df)
fe_df.head()

## Train the model

In this section, we focus on constructing and refining our predictive model. Initially, we divide our data, which is based on Weight of Evidence (WoE) features, into training and testing sets (`train_df`, `test_df`). With `lending_club.split`, we employ a simple random split, randomly allocating data points to each set to ensure a mix of examples in both. Additionally, by setting `add_constant=True`, we include an intercept term in our model.



In [ ]:
train_df, test_df = lending_club.split(fe_df, add_constant=True)

x_train = train_df.drop(lending_club.target_column, axis=1)
y_train = train_df[lending_club.target_column]
x_test = test_df.drop(lending_club.target_column, axis=1)
y_test = test_df[lending_club.target_column]

# Define the model
model = sm.GLM(
    y_train, 
    x_train, 
    family=sm.families.Binomial())

# Fit the model
model = model.fit()
model.summary()

### Compute probabilities

In [ ]:
train_probabilities = model.predict(x_train)
test_probabilities = model.predict(x_test)

### Compute binary predictions

In [ ]:
cut_off_threshold = 0.5
train_binary_predictions = (train_probabilities > cut_off_threshold).astype(int)
test_binary_predictions = (test_probabilities > cut_off_threshold).astype(int)

### Compute scores

In this phase, we translate model predictions into actionable scores using probability estimates generated by our trained model. 

In [ ]:
# Compute scores from the probabilities 
train_scores = lending_club.compute_scores(train_probabilities)
test_scores = lending_club.compute_scores(test_probabilities)

## Document the model

As part of documenting the model with the ValidMind Developer Framework, you need to preprocess the raw dataset, initialize some training and test datasets, initialize a model object you can use for testing, and then run the full suite of tests.

### Initialize the ValidMind datasets

Before you can run tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset` — the dataset that you want to provide as input to tests
- `input_id` - a unique identifier that allows tracking what inputs are used when running each individual test
- `target_column` — a required argument if tests require access to true values. This is the name of the target column in the dataset

With all datasets ready, you can now initialize the raw, processed, training and test datasets (`raw_df`, `preprocessed_df`, `fe_df`,  `train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):

In [ ]:
vm_raw_dataset = vm.init_dataset(
    dataset=df,
    input_id="raw_dataset",
    target_column=lending_club.target_column,
)

vm_preprocess_dataset = vm.init_dataset(
    dataset=preprocess_df,
    input_id="preprocess_dataset",
    target_column=lending_club.target_column,
)

vm_fe_dataset = vm.init_dataset(
    dataset=fe_df,
    input_id="fe_dataset",
    target_column=lending_club.target_column,
)

vm_train_ds = vm.init_dataset(
    dataset=train_df,
    input_id="train_dataset",
    target_column=lending_club.target_column,
)

vm_test_ds = vm.init_dataset(
    dataset=test_df, 
    input_id="test_dataset", 
    target_column=lending_club.target_column
)

### Initialize a model object

Additionally, you need to initialize a ValidMind model object (`vm_model`) that can be passed to other functions for analysis and tests on the data. You simply intialize this model object with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model):

In [ ]:
vm_model = vm.init_model(
    model,
    input_id="glm_model",
)

### Assign prediction values and probabilities to the datasets

With our model now trained, the next step involves assigning both predictive probabilities, which come directly from the model's predictions, and the binary prediction after applying the cutoff threshold described in the previous steps. This is achieved through the use of the `assign_predictions()` method associated with the VM `dataset` object. This method links the model's class prediction values and probabilities to our VM train and test datasets. 

In [ ]:
vm_train_ds.assign_predictions(
    model=vm_model,
    prediction_values=train_binary_predictions,
    prediction_probabilities = train_probabilities,
)

vm_test_ds.assign_predictions(
    model=vm_model,
    prediction_values=test_binary_predictions,
    prediction_probabilities = test_probabilities,
)

### Assign scores to the datasets

In the context of credit scorecards, we are interested in computing scores from model predictions, as scorecards fundamentally revolve around these scores rather than raw predictions. To make this process auditable and ensure scores are properly integrated with our datasets, we use the `add_extra_column()` method from the VM `dataset` object. This approach allows us to append scores directly to our data, maintaining a streamlined and coherent dataset ready for analysis.

In [ ]:
vm_train_ds.add_extra_column("glm_scores", train_scores)
vm_test_ds.add_extra_column("glm_scores", test_scores)

### Data validation

During data validation, we ensure that the data we're working with is accurate, consistent, and ready for empirical analysis. We're looking for any anomalies that might skew our models or results. In this section, we use tests to collect evidence across raw, preprocessed, and feature-engineered datasets.

#### Run tests for raw data tests

We perform initial validation of our raw data to understand its structure and detect any anomalies that could affect the modeling process. At this stage, we use a couple of tests that help us understand the data:

- Tabular Description Tables: statistical summary for each variable.
- Missing Values Bar Plot: plots presence of any missing data.

To ensure we can run this test again against a different dataset, we can label these test results with `<test_id>:raw_dataset`. This will make the results identifiable in the database, allowing us to include them in the UI

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.TabularDescriptionTables:raw_dataset",
    inputs = {
        "dataset": vm_raw_dataset
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.MissingValuesBarPlot:raw_dataset",
    inputs = {
        "dataset": vm_raw_dataset
    }
)
test.log()

#### Run tests for preprocessed data

Before the modeling process begins, the data undergoes a final quality validation. To esnure the data is ready for the next data preparation step, wWe conduct the following tests:

- Tabular Description Tables: Checking structural integrity after preprocessing.
- IQR Outliers Table: Searching for statistical anomalies that may impact results.
- Class Imbalance: Investigating the proportionality of outcome classes.
- Tabular Numerical Histograms: Visualizing distributions for numerical variables.
- Tabular Categorical Bar Plots: Analyzing frequencies of categorical variables.
- Target Rate Bar Plots: Examining the distribution of the target variable across categories.
- Pearson Correlation Matrix: Identifying linear relationships between variables.

As we did in the previous step, we now want to run the same description tables against the preprocessed dataset to identify improvements and observe the effects of our changes on the data. This time, we do this by adding the `<test_id>:preprocess_dataset>` label to our results.


In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.TabularDescriptionTables:preprocess_dataset",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.IQROutliersTable",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.ClassImbalance",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.TabularNumericalHistograms",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.TabularCategoricalBarPlots",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.TargetRateBarPlots",
    inputs = {
        "dataset": vm_preprocess_dataset
    },
    params = {
        "default_column": lending_club.target_column,
        "columns": None
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.PearsonCorrelationMatrix",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

#### Run tests for WoE analysis

To ensure data segmentation aligns with predictive value, we perform detailed tests on the following:

- WOE Bin Table: This test checks the bins and their corresponding Weight of Evidence values, ensuring accuracy and relevance.
- WOE Bin Plots: These plots display the distribution of WoE across the data spectrum, providing a visual assessment of alignment and consistency.

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.WOEBinTable",
    inputs = {
        "dataset": vm_preprocess_dataset
    },
    params = {
        "breaks_adj": lending_club.breaks_adj
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.WOEBinPlots",
    inputs = {
        "dataset": vm_preprocess_dataset
    },
    params = {
        "breaks_adj": lending_club.breaks_adj
    }
)
test.log()

### Model validation

This phase verifies the model's predictive power and generalizability. Tests are applied to both training and evaluation stages to ensure robustness and accuracy.

#### Run tests for model training

After training our model, one of the first tasks we want to perform is to inspect the model coefficients and some of their statistical properties, such as p-values. This is a crucial step as it informs us about the relevance of the features used to train the model. At this stage, we can begin making decisions regarding the quality of our model. These few tests assist us with this task:

- Regression Coeffs Plot: Visual inspection of the model's coefficients.
- Regression Models Coeffs: Detailed evaluation of the regression coefficients' values.

Note that we use the `models` interface and pass a model as a list. This is because these tests support multiple models. Therefore, if you have candidate models, you can use these tests to examine the differences in coefficients across multiple models.

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.RegressionCoeffsPlot",
    inputs = {
        "models": [vm_model]
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.RegressionModelsCoeffs",
    inputs = {
        "models": [vm_model]
    }
)
test.log()

#### Run tests for model evaluation

Once we are somewhat satisfied with the trained model after inspecting the coefficients, we elevate our evaluation by examining more detailed performance metrics, such as the confusion matrix, Gini coefficient, and various visualizations. At this stage, we also begin to examine the credit scores, comparing them across the test and training datasets to see how effectively they distinguish between defaulted and non-defaulted customers. 

These tests will help us to check whether the calibration of the scorecard is fit for purpose. Below, we list the tests we use to perform a thorough evaluation of both the binary classification model and the scorecard: 

- Classifier Performance: Summarizing logistic regression metrics.
- GINI Table: Assessing the discriminatory power of the model.
- Confusion Matrix: Understanding classification accuracy.
- ROC Curve: Plotting the trade-off between sensitivity and specificity.
- Prediction Probabilities Histogram: Distributing predictions to visualize outcomes.
- Cumulative Prediction Probabilities: Cumulative probability analysis for predictions.
- Scorecard Histogram: Evaluating the distribution of scorecard points.

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.sklearn.ClassifierPerformance:train_dataset",
    inputs = {
        "dataset": vm_train_ds,
        "model": vm_model,
    }
)

In [ ]:
test= vm.tests.run_test(
    "validmind.model_validation.sklearn.ClassifierPerformance:test_dataset",
    inputs = {
        "dataset": vm_test_ds,
        "model": vm_model,
    }
)

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.GINITable",
    inputs = {
        "datasets": [vm_train_ds, vm_test_ds],
        "model": vm_model,
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.sklearn.ConfusionMatrix",
    inputs = {
        "dataset": vm_test_ds,
        "model": vm_model
    },
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.sklearn.ROCCurve",
    inputs = {
        "model": vm_model,
        "dataset": vm_test_ds
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.PredictionProbabilitiesHistogram",
    inputs = {
        "model": vm_model,
        "datasets": [vm_train_ds, vm_test_ds]
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.CumulativePredictionProbabilities",
    inputs = {
        "model": vm_model,
        "datasets": [vm_train_ds, vm_test_ds]
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.ScorecardHistogram",
    inputs = {
        "model": vm_model,
        "datasets": [vm_train_ds, vm_test_ds]
    },
    params = {
        "score_column": "glm_scores"
    }
)
test.log()

## Next steps

You can look at the results of this test suite right in the notebook where you ran the code, as you would expect. But there is a better way: view the test results as part of your model documentation right in the ValidMind Platform UI:

1. In the [Platform UI](https://app.prod.validmind.ai), go to the **Documentation** page for the model you registered earlier.

2. Expand the following sections and take a look around:

   - **2. Data Preparation**
   - **3. Model Development**

What you can see now is a much more easily consumable version of the documentation, including the results of the tests you just performed, along with other parts of your model documentation that still need to be completed. There is a wealth of information that gets uploaded when you run the full test suite, so take a closer look around, especially at test results that might need attention (hint: some of the tests in *2.3. Feature Selection and Engineering* look like they need some attention).

If you want to learn more about where you are in the model documentation process, take a look at [How do I use the framework?](https://docs.validmind.ai/guide/get-started-developer-framework.html#how-do-i-use-the-framework).